This script demonstrates how to implement a basic character-level
sequence-to-sequence model. We apply it to translating
short English sentences into short French sentences,
character-by-character. Note that it is fairly unusual to
do character-level machine translation, as word-level
models are more common in this domain.

>this tutorial followed - https://github.com/keras-team/keras/blob/master/examples/lstm_seq2seq.py

In [2]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np


Using TensorFlow backend.


In [3]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'fra.txt'


In [4]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We use "\t" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [8]:
print(input_texts[:10]  , target_texts[:10])

['Go.', 'Hi.', 'Run!', 'Run!', 'Who?', 'Wow!', 'Fire!', 'Help!', 'Jump.', 'Stop!'] ['\tVa !\n', '\tSalut !\n', '\tCours\u202f!\n', '\tCourez\u202f!\n', '\tQui ?\n', '\tÇa alors\u202f!\n', '\tAu feu !\n', "\tÀ l'aide\u202f!\n", '\tSaute.\n', '\tÇa suffit\u202f!\n']


In [9]:
print(input_characters,target_characters)

{'p', '1', '2', 'M', ':', 'G', 'v', 'h', 'J', '%', "'", ' ', '6', 'S', 'O', 'R', 'C', 'g', 'f', 'V', 'P', 'o', 'F', 'a', 'N', 'Y', 'W', 'r', 'd', 'j', '$', 'H', ',', 'E', '8', 'i', '!', 'U', '7', 'n', 'l', 'B', '&', 'I', 'D', 'q', 'c', 'm', 'x', 'L', '3', 'K', 's', 'u', '-', 'y', '?', '5', 'w', 'k', 'z', 'A', 'Q', '0', '.', 'T', '9', 'b', 't', 'e'} {'J', '%', "'", 'S', 'O', 'o', '\t', 'r', 'j', '’', ',', 'E', '!', 'n', 'l', 'B', '\n', 'y', '?', '0', 'É', 'T', 'Ç', 't', 'e', '1', 'î', 'â', 'f', '\xa0', 'P', 'Y', 'a', '-', 'N', 'F', '8', 'i', '«', 'q', 'L', 'è', '5', 'k', 'ç', '9', 'û', 'é', 'À', 'M', 'v', 'G', 'h', '(', 'g', 'V', '\u202f', 'd', 'H', 'U', 'I', 'D', 'ë', 'c', 'm', 'x', '3', 's', 'u', 'à', '.', 'Ê', 'p', '2', ':', 'ô', '\u2009', ' ', 'C', 'R', 'ê', ')', '$', '&', '»', 'K', 'Q', 'A', 'ù', 'œ', 'ï', 'b', 'z'}


In [10]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])


In [11]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 70
Number of unique output tokens: 93
Max sequence length for inputs: 16
Max sequence length for outputs: 59


till now we found all the characters ,   
we have the dataset in lists,  
we have max len of sentence in english and french   
these all are vital for the model  

In [12]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])
#each character is assigned a number for encoding and decoding process

Turn the sentences into 3 Numpy arrays, encoder_input_data, decoder_input_data, decoder_target_data:  
1.encoder_input_data is a 3D array of shape (num_pairs, max_english_sentence_length, num_english_characters) containing a one-hot vectorization of the English sentences.  
2.decoder_input_data is a 3D array of shape (num_pairs, max_french_sentence_length, num_french_characters) containg a one-hot vectorization of the French sentences.  
3.decoder_target_data is the same as decoder_input_data but offset by one timestep. decoder_target_data[:, t, :] will be the same as decoder_input_data[:, t + 1, :].  

In [13]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [14]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [15]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens)) #num_encoder_tokens = no of unique characters(70)
encoder = LSTM(latent_dim, return_state=True)            #latent dim = 256
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

Instructions for updating:
Colocations handled automatically by placer.


In [24]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)       #?

In [25]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')


Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 29s 4ms/step - loss: 0.9031 - val_loss: 0.9118
Epoch 2/100
8000/8000 [==============================] - 26s 3ms/step - loss: 0.7116 - val_loss: 0.7431
Epoch 3/100
8000/8000 [==============================] - 45s 6ms/step - loss: 0.6075 - val_loss: 0.6656
Epoch 4/100
8000/8000 [==============================] - 33s 4ms/step - loss: 0.5533 - val_loss: 0.6227
Epoch 5/100
8000/8000 [==============================] - 34s 4ms/step - loss: 0.5155 - val_loss: 0.5882
Epoch 6/100
8000/8000 [==============================] - 36s 4ms/step - loss: 0.4851 - val_loss: 0.5578
Epoch 7/100
8000/8000 [==============================] - 28s 3ms/step - loss: 0.4597 - val_loss: 0.5388
Epoch 8/100
8000/8000 [==============================] - 29s 4ms/step - loss: 0.4373 - val_loss: 0.5160
Epoch 9/100
8000/8000 [==============================] - 47s 6ms/step - loss: 0.4180 - val_loss: 0.5033
Epoch 10/100
800

8000/8000 [==============================] - 25s 3ms/step - loss: 0.0625 - val_loss: 0.6946
Epoch 94/100
8000/8000 [==============================] - 25s 3ms/step - loss: 0.0619 - val_loss: 0.6999
Epoch 95/100
8000/8000 [==============================] - 23s 3ms/step - loss: 0.0607 - val_loss: 0.6960
Epoch 96/100
8000/8000 [==============================] - 26s 3ms/step - loss: 0.0601 - val_loss: 0.7064
Epoch 97/100
8000/8000 [==============================] - 27s 3ms/step - loss: 0.0592 - val_loss: 0.7131
Epoch 98/100
8000/8000 [==============================] - 23s 3ms/step - loss: 0.0583 - val_loss: 0.7123
Epoch 99/100
8000/8000 [==============================] - 23s 3ms/step - loss: 0.0579 - val_loss: 0.7147
Epoch 100/100
8000/8000 [==============================] - 25s 3ms/step - loss: 0.0571 - val_loss: 0.7153


C:\Users\Z0042WNT\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\engine\network.py:877: UserWarning: Layer lstm_3 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


sampling

In [26]:

# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)


In [27]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


In [28]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [29]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Go.
Decoded sentence: Va !

-
Input sentence: Hi.
Decoded sentence: Salut !

-
Input sentence: Run!
Decoded sentence: Cours !

-
Input sentence: Run!
Decoded sentence: Cours !

-
Input sentence: Who?
Decoded sentence: Qui ?

-
Input sentence: Wow!
Decoded sentence: Ça alors !

-
Input sentence: Fire!
Decoded sentence: Couvrez-le.

-
Input sentence: Help!
Decoded sentence: Salut !

-
Input sentence: Jump.
Decoded sentence: Saute.

-
Input sentence: Stop!
Decoded sentence: Stop !

-
Input sentence: Stop!
Decoded sentence: Stop !

-
Input sentence: Stop!
Decoded sentence: Stop !

-
Input sentence: Wait!
Decoded sentence: Attendez !

-
Input sentence: Wait!
Decoded sentence: Attendez !

-
Input sentence: Go on.
Decoded sentence: Poursuivez.

-
Input sentence: Go on.
Decoded sentence: Poursuivez.

-
Input sentence: Go on.
Decoded sentence: Poursuivez.

-
Input sentence: Hello!
Decoded sentence: Salut !

-
Input sentence: Hello!
Decoded sentence: Salut !

-
Input sentence: 